In [2]:
pip install imblearn

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 16.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

s3= boto3.resource('s3')
bucket_name= 'morgan-gant-data448-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'Iris.csv'

bucket_object= bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datefile
iris = pd.read_csv(file_content_stream)
iris.head()

ModuleNotFoundError: No module named 'imblearn'

In [7]:
#Frequency Table
iris['Species'].value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: Species, dtype: int64

In [42]:
#Changing labels to numbers
iris['Species_numb']= np.where(iris['Species'] == 'Iris-virginica', 1, np.where(iris['Species'] == 'Iris-versicolor', 2, 3))

In [43]:
#Defining input and target variables
x = iris.drop(columns=['Id', 'Species', 'Species_numb'], axis=1)
y= iris['Species_numb']

#Splitting the data
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size= .2, stratify=y)

In [44]:
#Scaling the data
scaler= MinMaxScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.fit_transform(x_test)

Random Forest

In [45]:
#Building model
one_vs_all_rf= OneVsRestClassifier(estimator= RandomForestClassifier(n_estimators=500, max_depth= 3)).fit(x_train, y_train)

#Predicting on the test
one_vs_all_rf_pred= one_vs_all_rf.predict_proba(x_test)
one_vs_all_rf_pred= np.argmax(one_vs_all_rf_pred, axis=1)+1

print(confusion_matrix(y_test, one_vs_all_rf_pred))
print(classification_report(y_test, one_vs_all_rf_pred))

[[10  0  0]
 [ 2  8  0]
 [ 0  0 10]]
              precision    recall  f1-score   support

           1       0.83      1.00      0.91        10
           2       1.00      0.80      0.89        10
           3       1.00      1.00      1.00        10

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30



Support Vector Machine

In [48]:
#Building model
one_vs_all_svm= OneVsRestClassifier(estimator= SVC(kernel='rbf', C=.1, probability=True)).fit(x_train, y_train)

#Predicting on the test
one_vs_all_svm_pred= one_vs_all_svm.predict_proba(x_test)
one_vs_all_svm_pred= np.argmax(one_vs_all_svm_pred, axis=1) + 1

print(confusion_matrix(y_test, one_vs_all_svm_pred))
print(classification_report(y_test, one_vs_all_svm_pred))

[[ 9  1  0]
 [ 1  9  0]
 [ 0  0 10]]
              precision    recall  f1-score   support

           1       0.90      0.90      0.90        10
           2       0.90      0.90      0.90        10
           3       1.00      1.00      1.00        10

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30



In [ ]:
#Based on the models above, with the results were produced from the split, either model could be used to predict iris species!